In [12]:
#%pip install --use-feature=2020-resolver --user torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#%pip install --use-feature=2020-resolver --upgrade --user kagglehub transformers datasets evaluate accelerate typing_extensions tqdm
#%pip install --use-feature=2020-resolver --user fastapi uvicorn pydantic

In [3]:
import sys
import os
import kagglehub
import pandas as pd
from datasets import Dataset, DatasetDict

def get_kaggle_df():
    path = kagglehub.dataset_download("zeyadkhalid/mbti-personality-types-500-dataset")
    return pd.read_csv(os.path.join(path,'MBTI 500.csv'))

def print_id2label():
    df = get_kaggle_df()
    l = list(df["type"].unique())
    print(f"id2label = {{ {','.join([ str(k) + ':' + repr(t) for k,t in enumerate(l)])}}}")
    print(f"label2id = {{ {','.join([ repr(t) + ':' +  str(k) for k,t in enumerate(l)])}}}")

def make_hf_ds():
    df = get_kaggle_df()

    def gen():
        for i,v in df.iterrows():
            yield {'text': v['posts'], 'label': label2id[v['type']]}

    ds = Dataset.from_generator(gen)
    ds_ = ds.train_test_split(test_size=0.1)
    ds_.save_to_disk(os.path.join('..','data','mbti'))

def load_ds():
    return DatasetDict.load_from_disk(os.path.join('..','data','mbti'))

In [5]:
#print_id2label()

In [4]:
id2label = { 0:'INTJ',1:'INTP',2:'ISFJ',3:'ISFP',4:'ISTJ',5:'ISTP',6:'ENFJ',7:'ENFP',8:'ENTJ',9:'ENTP',10:'ESFJ',11:'ESFP',12:'ESTJ',13:'ESTP',14:'INFJ',15:'INFP'}
label2id = { 'INTJ':0,'INTP':1,'ISFJ':2,'ISFP':3,'ISTJ':4,'ISTP':5,'ENFJ':6,'ENFP':7,'ENTJ':8,'ENTP':9,'ESFJ':10,'ESFP':11,'ESTJ':12,'ESTP':13,'INFJ':14,'INFP':15}
num_labels = 16

In [6]:
#make_hf_ds() 

In [5]:
ds = load_ds()

In [6]:
ds["train"][0]

{'text': 'bite like ni type know thing think would accurate say difficuy verbalize reason think process visual spatial holistic rather verbal sequential like answer include low function well high one especially notice one fjs fps way many people near middle scale sometimes score wrong side way achieve balance help ferret type imo see si se ni ne fi fe ti te think problem people introvert social anxiet society make social anxiety hate bother everyone way society whole treat introvert vouch poeple introvert well social anxiety sometimes social situation always easy figure social anxiety make want leave tire introversion sure social anxious introvert way hate people simply direct r socialanxiety simple oh gosh get much live family reason marry job enjoy generally thing great get little sometimes wish understand good maybe stand offish right word think get past prickly outside want overwhelm person since tend thing know month really appreciate insight anything say ohhh goodness must hard f

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_ds = ds.map(preprocess_function, batched=True)

In [9]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
import evaluate

accuracy = evaluate.load("accuracy")

In [11]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import torch
torch.cuda.is_available()

True

In [14]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2060'

In [15]:
training_args = TrainingArguments(
    output_dir="../models/my_mbti",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/5967 [00:00<?, ?it/s]

C:\Users\ruiba\AppData\Roaming\Python\Python38\site-packages\transformers\models\distilbert\modeling_distilbert.py:403: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 1.5981, 'grad_norm': 13.78458023071289, 'learning_rate': 1.8324115971174795e-05, 'epoch': 0.08}
{'loss': 0.9328, 'grad_norm': 9.150552749633789, 'learning_rate': 1.664823194234959e-05, 'epoch': 0.17}
{'loss': 0.8454, 'grad_norm': 9.421222686767578, 'learning_rate': 1.4972347913524386e-05, 'epoch': 0.25}
{'loss': 0.7677, 'grad_norm': 11.670267105102539, 'learning_rate': 1.329646388469918e-05, 'epoch': 0.34}
{'loss': 0.7372, 'grad_norm': 7.803522109985352, 'learning_rate': 1.1620579855873975e-05, 'epoch': 0.42}
{'loss': 0.6839, 'grad_norm': 11.943435668945312, 'learning_rate': 9.94469582704877e-06, 'epoch': 0.5}
{'loss': 0.6624, 'grad_norm': 6.102105617523193, 'learning_rate': 8.268811798223564e-06, 'epoch': 0.59}
{'loss': 0.64, 'grad_norm': 12.100564002990723, 'learning_rate': 6.592927769398358e-06, 'epoch': 0.67}
{'loss': 0.5919, 'grad_norm': 6.732193470001221, 'learning_rate': 4.917043740573153e-06, 'epoch': 0.75}
{'loss': 0.6071, 'grad_norm': 9.361861228942871, 'learning_rat

  0%|          | 0/663 [00:00<?, ?it/s]

{'eval_loss': 0.5660577416419983, 'eval_accuracy': 0.8261525407749599, 'eval_runtime': 200.8802, 'eval_samples_per_second': 52.803, 'eval_steps_per_second': 3.3, 'epoch': 1.0}
{'train_runtime': 6245.074, 'train_samples_per_second': 15.286, 'train_steps_per_second': 0.955, 'train_loss': 0.7746280895838571, 'epoch': 1.0}


TrainOutput(global_step=5967, training_loss=0.7746280895838571, metrics={'train_runtime': 6245.074, 'train_samples_per_second': 15.286, 'train_steps_per_second': 0.955, 'total_flos': 1.264849503879168e+16, 'train_loss': 0.7746280895838571, 'epoch': 1.0})

In [16]:
from transformers import pipeline
classifier = pipeline("text-classification", model="../models/my_mbti/checkpoint-5967")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [17]:
classifier(["I am a very happy person", "I am a very happy person"])

[{'label': 'INFJ', 'score': 0.13813404738903046},
 {'label': 'INFJ', 'score': 0.13813404738903046}]

In [19]:
from huggingface_hub import notebook_login
notebook_login()

Token has not been saved to git credential helper.


In [20]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/rbarreiro/my_mbti/commit/fed2d30358c31a8498d0528072127d2cd1765e0e', commit_message='End of training', commit_description='', oid='fed2d30358c31a8498d0528072127d2cd1765e0e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rbarreiro/my_mbti', endpoint='https://huggingface.co', repo_type='model', repo_id='rbarreiro/my_mbti'), pr_revision=None, pr_num=None)

In [22]:
from transformers import pipeline;classifier = pipeline("text-classification", model="rbarreiro/my_mbti")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
